In [1]:
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 90 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=60)

current_date=Timestamp('2023-12-30 20:00:00')


In [3]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    Modified version of the fetch_batch_raw_data
    """
    
    # fetching data for all concerned months using load_raw_data
    dates_to_fetch = pd.date_range(
        start=from_date, 
        end=to_date + pd.offsets.DateOffset(months=1), 
        freq='M'
    )
    
    rides = pd.DataFrame()
    for date in dates_to_fetch:
        date_ = date - pd.offsets.DateOffset(years=1)
        rides_date = load_raw_data(year=date_.year, months=date_.month)
        rides = pd.concat([rides, rides_date])


    # shift the data to pretend this is recent data 
    rides['pickup_datetime'] += pd.offsets.DateOffset(years=1)

    rides = rides[rides['pickup_datetime'] >= from_date]
    rides = rides[rides['pickup_datetime'] < to_date]

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [4]:
# from src.data import load_raw_data

# def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
#     """
#     Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
#     """
#     from_date_ = from_date - timedelta(days=7*52)
#     to_date_ = to_date - timedelta(days=7*52)
#     print(f'{from_date=}, {to_date_=}')

#     # download 2 files from website
#     rides = load_raw_data(year=from_date_.year, months=from_date_.month)
#     rides = rides[rides.pickup_datetime >= from_date_]
#     rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
#     rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

#     rides = pd.concat([rides, rides_2])

#     # shift the data to pretend this is recent data
#     rides['pickup_datetime'] += timedelta(days=7*52)

#     rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

#     return rides

In [5]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage


In [6]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 262/262 [00:00<00:00, 418.46it/s]


In [19]:
ts_data

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2023-10-31 20:00:00,0,1,1698782400000
1,2023-10-31 21:00:00,0,1,1698786000000
2,2023-10-31 22:00:00,0,1,1698789600000
3,2023-10-31 23:00:00,0,1,1698793200000
4,2023-11-01 00:00:00,0,1,1698796800000
...,...,...,...,...
377275,2023-12-30 15:00:00,2,265,1703948400000
377276,2023-12-30 16:00:00,12,265,1703952000000
377277,2023-12-30 17:00:00,6,265,1703955600000
377278,2023-12-30 18:00:00,4,265,1703959200000


In [17]:
print(f'unique pickup_hour: {ts_data.pickup_hour.nunique()}')
print(f'unique pickup_ts: {ts_data.pickup_ts.nunique()}')

unique pickup_hour: 1440
unique pickup_ts: 1440


In [8]:
# # string to datetime
# ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# # add column with Unix epoch milliseconds
# ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [20]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
# hopsworks.connection().close()

In [21]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/377280 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/305300/jobs/named/time_series_hourly_feature_group_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f2a13467cd0>, None)